<a href="https://colab.research.google.com/github/ShalokBalodi/Sentiment-analysis-/blob/main/Sentiment_Analysis_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title importing our packages
import pandas as pd   # Great for tables (google spreadsheets, microsoft excel, csv).
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import wordcloud
import os # Good for navigating your computer's files
import sys
pd.options.mode.chained_assignment = None #suppress warnings

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = spacy.load('en_core_web_md')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2023-11-11 12:02:38.199193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 12:02:38.199267: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 12:02:38.199316: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 12:02:42.953458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
#@title reading and exploring our dataset
data=pd.read_excel('/content/NLP_dataset.xlsx')
print(data.head())
print(len(data))


                                                text  stars
0  These lights are great; I like the longer adap...      5
1  This is another LED strip light that has a lot...      5
2  These lights are awesome! I have them in all 4...      5
3  This was even better than I imagined. My 10-ye...      5
4               An excellent product. Work properly.      5
763


## *Based on our visual understanding of the dataset, we need to convert stars into three catagories: positive, negative or neutral*

We define neutral as either:
*   "containing roughly equal positive and negative sentiments simultaneously " OR
*   "not specifing any sentiment towards the electric product for instance, just explaining the features nothing else"





In [5]:
def is_good_review(num_stars):
    if  num_stars>3:
        return 1
    elif num_stars==3:
        return 0
    else:
      return -1
# Change the stars column to either be 'good' or 'bad'.
data['is_good_review'] = data['stars'].apply(is_good_review)
data.head()

,text,stars,is_good_review
0,These lights are great; I like the longer adap...,5,1
1,This is another LED strip light that has a lot...,5,1
2,These lights are awesome! I have them in all 4...,5,1
3,This was even better than I imagined. My 10-ye...,5,1
4,An excellent product. Work properly.,5,1


Now we extract our input feature: "text" and output feature:"is_good_review"

In [6]:
#@markdown Now we create a function to tokenize our text datapoints as a part of pre-processing
X_text = data['text']
y = data['is_good_review']
#text cleaning function
def tokenize_vecs(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct):
          # -PRON- is a special all inclusive "lemma" spaCy uses for any pronoun, we want to exclude these
            clean_tokens.append(token)
    return clean_tokens

In [7]:
X_word2vec = []
for text in X_text:
  review = tokenize_vecs(text) # returns cleaned list of spacy tokens
  review_vec = [0]*300
  for word in review:
    review_vec += word.vector
  review_vec = review_vec / len(review)
  X_word2vec.append(review_vec)

X_word2vec = np.array(X_word2vec)


# Now, we

# 1.   Split our data into training and testing datasets
# 2.   Create a model to predict sentiment using Logistic Regression
# 3.   Train and test our model



In [11]:
w2v_model = LogisticRegression(max_iter = 2500)
# train-test split
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(X_word2vec, y, test_size=0.05, random_state=144)
w2v_model.fit(X_train_word2vec, y_train_word2vec)
w2v_preds = w2v_model.predict(X_test_word2vec)
accuracy = accuracy_score(y_test_word2vec, w2v_preds)
print (accuracy)

0.8205128205128205


Based on a relatively high accuracy of 82% on a testing sample of roughly 40
datapoints, we can correctly conjecture our model has not undergone overfitting and has correctly understood our data

In [34]:
#we now test our model on manually written data
text='would say it was just average'#@param{type:'string'}
X_text=[text]
X_word2vec = []
for text in X_text:
  review = tokenize_vecs(text) # returns cleaned list of spacy tokens
  review_vec = [0]*300
  for word in review:
    review_vec += word.vector
  review_vec = review_vec / len(review)
  X_word2vec.append(review_vec)


X_word2vec = np.array(X_word2vec)



In [35]:
preds = w2v_model.predict(X_word2vec)
print(preds)


[0]
